In [13]:
import os
import sys

import pandas as pd
import numpy as np

sys.path.insert(0, "/".join(os.path.abspath(os.curdir).split("/")[:-1:]))
from utils.utils import *

pd.set_option('future.no_silent_downcasting', True)

# Dados observatório educacional

## Carregar os dados

In [14]:
df = pd.read_csv("../output/merged_processed_general_data.csv", low_memory=False)

df = df[df['NO_MUNICIPIO'].isin(['Itabuna', 'Ilhéus'])]

df.head()

,NO_MUNICIPIO,ANO,TP_DEPENDENCIA,TP_CATEGORIA_ESCOLA_PRIVADA,TP_LOCALIZACAO,TP_LOCALIZACAO_DIFERENCIADA,TP_REGULAMENTACAO,TP_OCUPACAO_PREDIO_ESCOLAR,IN_PREDIO_COMPARTILHADO,IN_AGUA_FILTRADA,...,QT_MAT_INF_PRE_INT,QT_DOC_INF,QT_DOC_INF_CRE,QT_DOC_INF_PRE,QT_TUR_INF,QT_TUR_INF_CRE,QT_TUR_INF_PRE,IN_PROF_GESTAO,IN_PROF_COORDENADOR,QT_PROF_COORDENADOR
0,Ilhéus,2010,Estadual,Não aplicável para escolas públicas,Urbana,Escola não está em área de localização diferen...,Sim,Não aplicável para escolas que não ocupam préd...,Não,Sim,...,0,0,0,0,0,0,0,Não aplicável,Não aplicável,Não aplicável
1,Ilhéus,2010,Municipal,Não aplicável para escolas públicas,Urbana,Escola não está em área de localização diferen...,Sim,Não aplicável para escolas que não ocupam préd...,Não,Sim,...,0,0,0,0,0,0,0,Não aplicável,Não aplicável,Não aplicável
2,Ilhéus,2010,Municipal,Não aplicável para escolas públicas,Urbana,Escola não está em área de localização diferen...,Sim,Não aplicável para escolas que não ocupam préd...,Não,Sim,...,39,7,3,4,6,2,4,Não aplicável,Não aplicável,Não aplicável
3,Ilhéus,2010,Municipal,Não aplicável para escolas públicas,Urbana,Escola não está em área de localização diferen...,Em tramitação,Não aplicável para escolas que não ocupam préd...,Não,Sim,...,0,4,0,4,3,0,3,Não aplicável,Não aplicável,Não aplicável
4,Ilhéus,2010,Privada,Filantrópica,Urbana,Escola não está em área de localização diferen...,Sim,Não aplicável para escolas que não ocupam préd...,Não,Sim,...,0,3,2,1,3,2,1,Não aplicável,Não aplicável,Não aplicável


## Dados mais específicos da educação infantil dos municípios

In [15]:
columns = ['QT_MAT_BAS_0_3', 'QT_MAT_BAS_4_5', 'QT_MAT_INF', 'QT_DOC_INF']

df_especifico = df[['NO_MUNICIPIO', 'ANO'] + columns].copy()
df_especifico = replace_nao_aplicavel(df_especifico, columns)
df_especifico['QT_ESC'] = df_especifico['QT_MAT_INF'].apply(lambda x: 1)

df_especifico = df_especifico.groupby(by=["NO_MUNICIPIO", "ANO"], as_index=False).sum(numeric_only=True)

df_especifico.head()

,NO_MUNICIPIO,ANO,QT_MAT_BAS_0_3,QT_MAT_BAS_4_5,QT_MAT_INF,QT_DOC_INF,QT_ESC
0,Ilhéus,2010,952,3887,5220,336,123
1,Ilhéus,2011,887,3706,4953,309,111
2,Ilhéus,2012,889,3764,4956,341,135
3,Ilhéus,2013,931,3467,4685,319,140
4,Ilhéus,2014,1182,3503,4959,374,149


__Dados extraídos__:

- QT_MAT_BAS_0_3: População de 0 a 3 de idade escolar
- QT_MAT_BAS_4_5: População de 4 a 5 de idade escolar
- QT_MAT_INF: Número geral de matrículas
- QT_DOC_INF: Número geral de docentes
- QT_ESC: Número geral de escolas

## Média de alunos por turma na pré-escola e creche

In [16]:
# Média de alunos por turma
columns = ['QT_MAT_INF', 'QT_MAT_INF_CRE', 'QT_MAT_INF_PRE', 'QT_TUR_INF',
           'QT_TUR_INF_CRE', 'QT_TUR_INF_PRE']

df_mda_alu_tur = df[['NO_MUNICIPIO', 'ANO'] + columns].copy()
df_mda_alu_tur = replace_nao_aplicavel(df_mda_alu_tur, columns)

df_mda_alu_tur = df_mda_alu_tur.groupby(by=["NO_MUNICIPIO", "ANO"], as_index=False).sum(numeric_only=True)
df_mda_alu_tur['MDA_ALU_TUR_INF'] = get_mean_columns(df_mda_alu_tur, 'QT_MAT_INF', 'QT_TUR_INF')
df_mda_alu_tur['MDA_ALU_TUR_CRE'] = get_mean_columns(df_mda_alu_tur, 'QT_MAT_INF_CRE', 'QT_TUR_INF_CRE')
df_mda_alu_tur['MDA_ALU_TUR_PRE'] = get_mean_columns(df_mda_alu_tur, 'QT_MAT_INF_PRE', 'QT_TUR_INF_PRE')

# Média de alunos por turma segundo dependência administrativa
mean_columns = ['QT_MAT_INF', 'QT_TUR_INF']
tmp = get_mean_columns_pivoted(df, mean_columns, 'TP_DEPENDENCIA')
tmp.rename(columns={'Estadual': 'MDA_ALU_TUR_INF_EST', 'Federal': 'MDA_ALU_TUR_INF_FED',
                     'Municipal': 'MDA_ALU_TUR_INF_MUN', 'Privada': 'MDA_ALU_TUR_INF_PRI'}, inplace=True)

# Média de alunos por turma segundo localização
mean_columns = ['QT_MAT_INF', 'QT_TUR_INF']
tmp2 = get_mean_columns_pivoted(df, mean_columns, 'TP_LOCALIZACAO')
tmp2.rename(columns={'Urbana': 'MDA_ALU_TUR_INF_URB', 'Rural': 'MDA_ALU_TUR_INF_RUR'}, inplace=True)

df_mda_alu_tur = pd.merge(df_mda_alu_tur, tmp, on=['NO_MUNICIPIO', 'ANO'], how='left')
df_mda_alu_tur = pd.merge(df_mda_alu_tur, tmp2, on=['NO_MUNICIPIO', 'ANO'], how='left')
df_mda_alu_tur.drop(columns=['QT_MAT_INF_CRE', 'QT_TUR_INF_CRE', 'QT_MAT_INF_PRE',
                             'QT_TUR_INF_PRE'], inplace=True)
df_mda_alu_tur.head()

,NO_MUNICIPIO,ANO,QT_MAT_INF,QT_TUR_INF,MDA_ALU_TUR_INF,MDA_ALU_TUR_CRE,MDA_ALU_TUR_PRE,MDA_ALU_TUR_INF_EST,MDA_ALU_TUR_INF_FED,MDA_ALU_TUR_INF_MUN,MDA_ALU_TUR_INF_PRI,MDA_ALU_TUR_INF_RUR,MDA_ALU_TUR_INF_URB
0,Ilhéus,2010,5220,303,17.227723,15.160000,17.636364,12.888889,0.0,19.462585,15.258503,18.745455,16.891129
1,Ilhéus,2011,4953,289,17.138408,14.181818,17.669388,21.142857,0.0,19.569343,14.648276,20.208333,16.526971
2,Ilhéus,2012,4956,324,15.296296,12.304348,15.791367,13.000000,0.0,18.205674,13.058480,17.264151,14.911439
3,Ilhéus,2013,4685,318,14.732704,12.220339,15.305019,14.230769,0.0,17.592857,12.345455,17.807692,14.131579
4,Ilhéus,2014,4959,355,13.969014,11.364706,14.788889,12.133333,0.0,15.423313,12.785311,13.455882,14.090592


__Dados extraídos__:

- QT_MAT_INF: Número geral de matrículas
- QT_TUR_INF: Número geral de turmas
- MDA_ALU_TUR_INF: Média geral de alunos por turma
- MDA_ALU_TUR_INF_CRE: Média de alunos por turma em creches
- MDA_ALU_TUR_INF_PRE: Média de alunos por turma em pré-escolas
- MDA_ALU_TUR_INF_EST: Média de alunos por turma em escolas estaduais
- MDA_ALU_TUR_INF_FED: Média de alunos por turma em escolas federais
- MDA_ALU_TUR_INF_MUN: Média de alunos por turma em escolas municipais
- MDA_ALU_TUR_INF_PRI: Média de alunos por turma em escolas privadas
- MDA_ALU_TUR_INF_RUR: Média de alunos por turma em localidades rurais
- MDA_ALU_TUR_INF_URB: Média de alunos por turma em localidades urbanas

## Número de estabelecimentos de educação infantil (creches)

In [17]:
columns = ["QT_MAT_INF_CRE"]

df_estab_cre = df[["NO_MUNICIPIO", "ANO"] + columns].copy()

df_estab_cre = replace_nao_aplicavel(df_estab_cre, columns)
df_estab_cre["QT_ESC_INF_CRE"] = df_estab_cre["QT_MAT_INF_CRE"].apply(
    lambda x: 1 if x > 0 else 0
)

df_estab_cre = df_estab_cre.groupby(by=["NO_MUNICIPIO", "ANO"], as_index=False).sum(
    numeric_only=True
).drop(columns="QT_MAT_INF_CRE")

tmp = get_qt_estabelecimento_columns_pivoted(df, 'QT_MAT_INF_CRE', 'TP_DEPENDENCIA')
tmp.rename(columns={'Estadual': 'QT_ESC_INF_CRE_EST', 'Federal': 'QT_ESC_INF_CRE_FED',
                        'Municipal': 'QT_ESC_INF_CRE_MUN', 'Privada': 'QT_ESC_INF_CRE_PRI'}, inplace=True)

df_estab_cre = pd.merge(df_estab_cre, tmp, on=['NO_MUNICIPIO', 'ANO'], how='left')

# Número de pré-escolas por localização
tmp = get_qt_estabelecimento_columns_pivoted(df, 'QT_MAT_INF_CRE', 'TP_LOCALIZACAO')
tmp.rename(columns={'Urbana': 'QT_INF_CRE_URB', 'Rural': 'QT_INF_CRE_RUR'}, inplace=True)

df_estab_cre = pd.merge(df_estab_cre, tmp, on=['NO_MUNICIPIO', 'ANO'], how='left')
df_estab_cre.head()

,NO_MUNICIPIO,ANO,QT_ESC_INF_CRE,QT_ESC_INF_CRE_EST,QT_ESC_INF_CRE_FED,QT_ESC_INF_CRE_MUN,QT_ESC_INF_CRE_PRI,QT_INF_CRE_RUR,QT_INF_CRE_URB
0,Ilhéus,2010,20,0,0,8,12,3,17
1,Ilhéus,2011,19,0,0,6,13,3,16
2,Ilhéus,2012,22,0,0,7,15,3,19
3,Ilhéus,2013,24,0,0,7,17,3,21
4,Ilhéus,2014,35,0,0,15,20,7,28


__Dados extraídos__:

- QT_ESC_INF_CRE: Número geral de matrículas
- QT_ESC_INF_CRE_EST: Número de creches estaduais
- QT_ESC_INF_CRE_FED: Número de creches federais
- QT_ESC_INF_CRE_MUN: Número de creches municipais
- QT_ESC_INF_CRE_PRI: Número de creches privadas
- QT_ESC_INF_CRE_RUR: Número de creches em localidade rural
- QT_ESC_INF_CRE_URB: Número de creches em localidade urbana

## Número de estabelecimentos de educação infantil (pré-escola)

In [18]:
columns = ["QT_MAT_INF_PRE"]

df_estab_pre = df[["NO_MUNICIPIO", "ANO"] + columns].copy()
df_estab_pre = replace_nao_aplicavel(df_estab_pre, columns)
df_estab_pre["QT_INF_PRE"] = df_estab_pre["QT_MAT_INF_PRE"].apply(
    lambda x: 1 if x > 0 else 0
)
df_estab_pre = df_estab_pre.groupby(by=["NO_MUNICIPIO", "ANO"], as_index=False).sum(
    numeric_only=True
).drop(columns="QT_MAT_INF_PRE")

# Número de pré-escolas por dependência administrativa
tmp = get_qt_estabelecimento_columns_pivoted(df, 'QT_MAT_INF_PRE', 'TP_DEPENDENCIA')
tmp.rename(columns={'Estadual': 'QT_INF_PRE_EST', 'Federal': 'QT_INF_PRE_FED',
                        'Municipal': 'QT_INF_PRE_MUN', 'Privada': 'QT_INF_PRE_PRI'}, inplace=True)

df_estab_pre = pd.merge(df_estab_pre, tmp, on=['NO_MUNICIPIO', 'ANO'], how='left')

# Número de pré-escolas por localização
tmp = get_qt_estabelecimento_columns_pivoted(df, 'QT_MAT_INF_PRE', 'TP_LOCALIZACAO')
tmp.rename(columns={'Urbana': 'QT_INF_PRE_URB', 'Rural': 'QT_INF_PRE_RUR'}, inplace=True)

df_estab_pre = pd.merge(df_estab_pre, tmp, on=['NO_MUNICIPIO', 'ANO'], how='left')
df_estab_pre.head()

,NO_MUNICIPIO,ANO,QT_INF_PRE,QT_INF_PRE_EST,QT_INF_PRE_FED,QT_INF_PRE_MUN,QT_INF_PRE_PRI,QT_INF_PRE_RUR,QT_INF_PRE_URB
0,Ilhéus,2010,56,1,0,28,27,14,42
1,Ilhéus,2011,56,1,0,27,28,14,42
2,Ilhéus,2012,64,1,0,28,35,14,50
3,Ilhéus,2013,60,1,0,28,31,14,46
4,Ilhéus,2014,63,2,0,29,32,15,48


__Dados extraídos__:

- QT_INF_PRE: Número geral de matrículas
- QT_INF_PRE_EST: Número de pré-escolas estaduais
- QT_INF_PRE_FED: Número de pré-escolas federais
- QT_INF_PRE_MUN: Número de pré-escolas municipais
- QT_INF_PRE_PRI: Número de pré-escolas privadas
- QT_ESC_INF_PRE_RUR: Número de pré-escolas em localidade rural
- QT_ESC_INF_PRE_URB: Número de pré-escolas em localidade urbana

## Número de matrículas em creches

In [19]:
columns = ["QT_MAT_INF_CRE"]

df_mat_cre = df[["NO_MUNICIPIO", "ANO"] + columns].copy()
df_mat_cre = replace_nao_aplicavel(df_mat_cre, columns)
df_mat_cre = df_mat_cre.groupby(by=["NO_MUNICIPIO", "ANO"], as_index=False).sum(
    numeric_only=True
)
tmp = df[["NO_MUNICIPIO", "ANO", "QT_MAT_INF_CRE", "TP_DEPENDENCIA"]].copy()
tmp = replace_nao_aplicavel(tmp, ["QT_MAT_INF_CRE"])
tmp = tmp.groupby(by=["NO_MUNICIPIO", "ANO", "TP_DEPENDENCIA"], as_index=False).sum(
    numeric_only=True
)
tmp = tmp.pivot_table(index=["NO_MUNICIPIO", "ANO"], columns="TP_DEPENDENCIA",
                      values="QT_MAT_INF_CRE").reset_index()
tmp.fillna(0, inplace=True)
tmp[['Estadual', 'Federal', 'Municipal', 'Privada']] = tmp[['Estadual', 'Federal',
                                                            'Municipal', 'Privada']].astype(int)
tmp.rename(columns={'Estadual': 'QT_MAT_INF_CRE_EST', 'Federal': 'QT_MAT_INF_CRE_FED',
                    'Municipal': 'QT_MAT_INF_CRE_MUN', 'Privada': 'QT_MAT_INF_CRE_PRI'}, inplace=True)

df_mat_cre = pd.merge(df_mat_cre, tmp, on=['NO_MUNICIPIO', 'ANO'], how='left')
df_mat_cre.head()

,NO_MUNICIPIO,ANO,QT_MAT_INF_CRE,QT_MAT_INF_CRE_EST,QT_MAT_INF_CRE_FED,QT_MAT_INF_CRE_MUN,QT_MAT_INF_CRE_PRI
0,Ilhéus,2010,758,0,0,301,457
1,Ilhéus,2011,624,0,0,171,453
2,Ilhéus,2012,566,0,0,198,368
3,Ilhéus,2013,721,0,0,219,502
4,Ilhéus,2014,966,0,0,368,598


## Número de matrículas em pré-escolas

In [20]:
columns = ["QT_MAT_INF_PRE"]

df_mat_pre = df[["NO_MUNICIPIO", "ANO"] + columns].copy()
df_mat_pre = replace_nao_aplicavel(df_mat_pre, columns)
df_mat_pre = df_mat_pre.groupby(by=["NO_MUNICIPIO", "ANO"], as_index=False).sum(
    numeric_only=True
)
tmp = df[["NO_MUNICIPIO", "ANO", "QT_MAT_INF_PRE", "TP_DEPENDENCIA"]].copy()
tmp = replace_nao_aplicavel(tmp, ["QT_MAT_INF_PRE"])
tmp = tmp.groupby(by=["NO_MUNICIPIO", "ANO", "TP_DEPENDENCIA"], as_index=False).sum(
    numeric_only=True
)
tmp = tmp.pivot_table(index=["NO_MUNICIPIO", "ANO"], columns="TP_DEPENDENCIA",
                      values="QT_MAT_INF_PRE").reset_index()
tmp.fillna(0, inplace=True)
tmp[['Estadual', 'Federal', 'Municipal', 'Privada']] = tmp[['Estadual', 'Federal',
                                                            'Municipal', 'Privada']].astype(int)
tmp.rename(columns={'Estadual': 'QT_MAT_INF_PRE_EST', 'Federal': 'QT_MAT_INF_PRE_FED',
                    'Municipal': 'QT_MAT_INF_PRE_MUN', 'Privada': 'QT_MAT_INF_PRE_PRI'}, inplace=True)

df_mat_pre = pd.merge(df_mat_pre, tmp, on=['NO_MUNICIPIO', 'ANO'], how='left')
df_mat_pre.head()

,NO_MUNICIPIO,ANO,QT_MAT_INF_PRE,QT_MAT_INF_PRE_EST,QT_MAT_INF_PRE_FED,QT_MAT_INF_PRE_MUN,QT_MAT_INF_PRE_PRI
0,Ilhéus,2010,4462,116,0,2560,1786
1,Ilhéus,2011,4329,148,0,2510,1671
2,Ilhéus,2012,4390,156,0,2369,1865
3,Ilhéus,2013,3964,185,0,2244,1535
4,Ilhéus,2014,3993,182,0,2146,1665


## Infraestrutura

In [21]:
columns = ['IN_INF_PRE', 'IN_INF_CRE', 'IN_AGUA_FILTRADA', 'IN_AGUA_POTAVEL', 'IN_BANHEIRO_PNE',
           'IN_BANHEIRO_EI', 'IN_BIBLIOTECA', 'IN_LIXO_SERVICO_COLETA', 'IN_COZINHA',
           'IN_REFEITORIO', 'IN_PARQUE_INFANTIL', 'IN_QUADRA_ESPORTES',
           'IN_SALA_DIRETORIA', 'IN_BIBLIOTECA_SALA_LEITURA', 'IN_SALA_PROFESSOR',
           'IN_ENERGIA_INEXISTENTE', "IN_AGUA_INEXISTENTE", "IN_ESGOTO_INEXISTENTE"]
columns_to_drop = columns

df_infra = df[['NO_MUNICIPIO', 'ANO'] + columns].copy()
df_infra = replace_sim_nao_to_binary(df_infra, columns)

df_infra['QT_ESC'] = df_infra['IN_INF_PRE'].apply(lambda x: 1)
df_infra['QT_ESC_INF_PRE'] = df_infra['IN_INF_CRE']
df_infra['QT_ESC_INF_CRE'] = df_infra['IN_INF_PRE']
columns_to_drop += ['QT_ESC', 'QT_ESC_INF_PRE', 'QT_ESC_INF_CRE']

water_qualities = ['POTAVEL', 'FILTRADA']
bathroom_types = ['PNE', 'EI']

for water_quality in water_qualities:
    df_infra[f'IN_AGUA_{water_quality}_CRE'] = df_infra.apply(
        lambda x: 1 if x[f'IN_AGUA_{water_quality}'] == 1
        and x['IN_INF_CRE'] == 1 else 0, axis=1
    )
    df_infra[f'IN_AGUA_{water_quality}_PRE'] = df_infra.apply(
        lambda x: 1 if x[f'IN_AGUA_{water_quality}'] == 1
        and x['IN_INF_PRE'] == 1 else 0, axis=1
    )

for bathroom_type in bathroom_types:
    df_infra[f'IN_BANHEIRO_{bathroom_type}_CRE'] = df_infra.apply(
        lambda x: 1 if x[f'IN_BANHEIRO_{bathroom_type}'] == 1
        and x['IN_INF_CRE'] == 1 else 0, axis=1
    )
    df_infra[f'IN_BANHEIRO_{bathroom_type}_PRE'] = df_infra.apply(
        lambda x: 1 if x[f'IN_BANHEIRO_{bathroom_type}'] == 1
        and x['IN_INF_PRE'] == 1 else 0, axis=1
    )

df_infra = df_infra.groupby(by=["NO_MUNICIPIO", "ANO"], as_index=False).sum(numeric_only=True)
df_infra.head()

,NO_MUNICIPIO,ANO,IN_INF_PRE,IN_INF_CRE,IN_AGUA_FILTRADA,IN_AGUA_POTAVEL,IN_BANHEIRO_PNE,IN_BANHEIRO_EI,IN_BIBLIOTECA,IN_LIXO_SERVICO_COLETA,...,QT_ESC_INF_PRE,QT_ESC_INF_CRE,IN_AGUA_POTAVEL_CRE,IN_AGUA_POTAVEL_PRE,IN_AGUA_FILTRADA_CRE,IN_AGUA_FILTRADA_PRE,IN_BANHEIRO_PNE_CRE,IN_BANHEIRO_PNE_PRE,IN_BANHEIRO_EI_CRE,IN_BANHEIRO_EI_PRE
0,Ilhéus,2010,56,20,95,0,25,17,42,90,...,20,56,0,0,20,56,5,14,7,17
1,Ilhéus,2011,56,19,100,0,29,22,43,97,...,19,56,0,0,19,55,3,13,8,19
2,Ilhéus,2012,64,22,108,0,43,28,51,106,...,22,64,0,0,22,63,4,18,8,24
3,Ilhéus,2013,60,24,102,0,42,27,43,99,...,24,60,0,0,24,58,5,18,12,24
4,Ilhéus,2014,63,35,107,0,47,32,44,104,...,35,63,0,0,35,62,10,22,19,29


In [22]:
general_columns_prop = ['BIBLIOTECA', 'LIXO_SERVICO_COLETA', 'REFEITORIO',
                        'COZINHA', 'PARQUE_INFANTIL', 'QUADRA_ESPORTES', 'SALA_DIRETORIA',
                        'BIBLIOTECA_SALA_LEITURA', 'SALA_PROFESSOR', 'ENERGIA_INEXISTENTE',
                        "AGUA_INEXISTENTE", "ESGOTO_INEXISTENTE"]

for column in general_columns_prop:
    df_infra[f'PROP_{column}'] = get_prop_columns(df_infra, f'IN_{column}', 'QT_ESC')
    df_infra[f'PROP_{column}_INF_CRE'] = get_prop_columns(df_infra, f'IN_{column}', 'QT_ESC_INF_CRE')
    df_infra[f'PROP_{column}_INF_PRE'] = get_prop_columns(df_infra, f'IN_{column}', 'QT_ESC_INF_PRE')

    columns_to_drop += [f'IN_{column}']

for water_quality in water_qualities:
    df_infra[f'PROP_AGUA_{water_quality}'] = get_prop_columns(
        df_infra,f'IN_AGUA_{water_quality}', 'QT_ESC'
        )
    df_infra[f'PROP_AGUA_{water_quality}_INF_CRE'] = get_prop_columns(
        df_infra, f'IN_AGUA_{water_quality}_CRE', 'QT_ESC_INF_PRE'
        )
    df_infra[f'PROP_AGUA_{water_quality}_INF_PRE'] = get_prop_columns(
        df_infra, f'IN_AGUA_{water_quality}_PRE', 'QT_ESC_INF_CRE'
        )

    columns_to_drop += [f'IN_AGUA_{water_quality}_CRE', f'IN_AGUA_{water_quality}_PRE']

for bathroom_type in bathroom_types:
    df_infra[f'PROP_BANHEIRO_{bathroom_type}'] = get_prop_columns(
        df_infra,f'IN_BANHEIRO_{bathroom_type}', 'QT_ESC'
        )
    df_infra[f'PROP_BANHEIRO_{bathroom_type}_INF_CRE'] = get_prop_columns(
        df_infra, f'IN_BANHEIRO_{bathroom_type}_CRE', 'QT_ESC_INF_PRE'
        )
    df_infra[f'PROP_BANHEIRO_{bathroom_type}_INF_PRE'] = get_prop_columns(
        df_infra, f'IN_BANHEIRO_{bathroom_type}_PRE', 'QT_ESC_INF_CRE'
        )

    columns_to_drop += [f'IN_BANHEIRO_{bathroom_type}_CRE', f'IN_BANHEIRO_{bathroom_type}_PRE']

df_infra.drop(columns=columns_to_drop, inplace=True)

df_infra.head()

,NO_MUNICIPIO,ANO,PROP_BIBLIOTECA,PROP_BIBLIOTECA_INF_CRE,PROP_BIBLIOTECA_INF_PRE,PROP_LIXO_SERVICO_COLETA,PROP_LIXO_SERVICO_COLETA_INF_CRE,PROP_LIXO_SERVICO_COLETA_INF_PRE,PROP_REFEITORIO,PROP_REFEITORIO_INF_CRE,...,PROP_AGUA_POTAVEL_INF_PRE,PROP_AGUA_FILTRADA,PROP_AGUA_FILTRADA_INF_CRE,PROP_AGUA_FILTRADA_INF_PRE,PROP_BANHEIRO_PNE,PROP_BANHEIRO_PNE_INF_CRE,PROP_BANHEIRO_PNE_INF_PRE,PROP_BANHEIRO_EI,PROP_BANHEIRO_EI_INF_CRE,PROP_BANHEIRO_EI_INF_PRE
0,Ilhéus,2010,0.34,0.75,0.0,0.73,0.0,0.0,0.00,0.00,...,0.0,0.77,1.0,1.00,0.20,0.25,0.25,0.14,0.35,0.30
1,Ilhéus,2011,0.39,0.77,0.0,0.87,0.0,0.0,0.00,0.00,...,0.0,0.90,1.0,0.98,0.26,0.16,0.23,0.20,0.42,0.34
2,Ilhéus,2012,0.38,0.80,0.0,0.79,0.0,0.0,0.16,0.34,...,0.0,0.80,1.0,0.98,0.32,0.18,0.28,0.21,0.36,0.38
3,Ilhéus,2013,0.31,0.72,0.0,0.71,0.0,0.0,0.14,0.32,...,0.0,0.73,1.0,0.97,0.30,0.21,0.30,0.19,0.50,0.40
4,Ilhéus,2014,0.30,0.70,0.0,0.70,0.0,0.0,0.17,0.40,...,0.0,0.72,1.0,0.98,0.32,0.29,0.35,0.21,0.54,0.46


## Juntar e salvar dados

In [23]:
df_full = pd.merge(df_especifico, df_mda_alu_tur, on=['NO_MUNICIPIO', 'ANO'], how='left', suffixes=('', '_y'))
df_full = pd.merge(df_full, df_estab_cre, on=['NO_MUNICIPIO', 'ANO'], how='left', suffixes=('', '_y'))
df_full = pd.merge(df_full, df_estab_pre, on=['NO_MUNICIPIO', 'ANO'], how='left', suffixes=('', '_y'))
df_full = pd.merge(df_full, df_mat_cre, on=['NO_MUNICIPIO', 'ANO'], how='left', suffixes=('', '_y'))
df_full = pd.merge(df_full, df_mat_pre, on=['NO_MUNICIPIO', 'ANO'], how='left', suffixes=('', '_y'))
df_full = pd.merge(df_full, df_infra, on=['NO_MUNICIPIO', 'ANO'], how='left', suffixes=('', '_y'))

columns = df_full.columns
columns = [column for column in columns if '_y' not in column]
df_full = df_full[columns]

df_full.head()

,NO_MUNICIPIO,ANO,QT_MAT_BAS_0_3,QT_MAT_BAS_4_5,QT_MAT_INF,QT_DOC_INF,QT_ESC,QT_TUR_INF,MDA_ALU_TUR_INF,MDA_ALU_TUR_CRE,...,PROP_AGUA_POTAVEL_INF_PRE,PROP_AGUA_FILTRADA,PROP_AGUA_FILTRADA_INF_CRE,PROP_AGUA_FILTRADA_INF_PRE,PROP_BANHEIRO_PNE,PROP_BANHEIRO_PNE_INF_CRE,PROP_BANHEIRO_PNE_INF_PRE,PROP_BANHEIRO_EI,PROP_BANHEIRO_EI_INF_CRE,PROP_BANHEIRO_EI_INF_PRE
0,Ilhéus,2010,952,3887,5220,336,123,303,17.227723,15.160000,...,0.0,0.77,1.0,1.00,0.20,0.25,0.25,0.14,0.35,0.30
1,Ilhéus,2011,887,3706,4953,309,111,289,17.138408,14.181818,...,0.0,0.90,1.0,0.98,0.26,0.16,0.23,0.20,0.42,0.34
2,Ilhéus,2012,889,3764,4956,341,135,324,15.296296,12.304348,...,0.0,0.80,1.0,0.98,0.32,0.18,0.28,0.21,0.36,0.38
3,Ilhéus,2013,931,3467,4685,319,140,318,14.732704,12.220339,...,0.0,0.73,1.0,0.97,0.30,0.21,0.30,0.19,0.50,0.40
4,Ilhéus,2014,1182,3503,4959,374,149,355,13.969014,11.364706,...,0.0,0.72,1.0,0.98,0.32,0.29,0.35,0.21,0.54,0.46


In [24]:
with open ('../output/dicionario_dados_gerais.txt', 'r') as fd:
    dicionario_dados = {'Sigla': [], 'Descrição': []}

    for line in fd.readlines():
        acronym, description = line.split(' ', 1)
        description = description.replace('\n', '')

        dicionario_dados['Sigla'].append(acronym)
        dicionario_dados['Descrição'].append(description)


dicionario_dados_df = pd.DataFrame(dicionario_dados)

with pd.ExcelWriter("../output/full_data.xlsx") as writer:
    df_full.to_excel(writer, index=False, sheet_name="dados_gerais")
    dicionario_dados_df.to_excel(writer, index=False, sheet_name="dicionario_dados")